In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/22 18:04:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
spark.createDataFrame([(1,"test"),(2,'test')],schema=' value int, name string ').show()

+-----+----+
|value|name|
+-----+----+
|    1|test|
|    2|test|
+-----+----+



In [ ]:
CONFLUENT_IP = ""
KAFKA_SERVER = f"{CONFLUENT_IP}:9092"
SCHEMA_REGISTRY_URL = f"http://{CONFLUENT_IP}:8081"
TOPIC = "ongoing-trips"

In [ ]:
import requests
import json

from pprint import pprint

# On récupère les versions du sujet ongoing-trips-value
rep_schema_versions = requests.get(
    SCHEMA_REGISTRY_URL + f"/subjects/ongoing-trips-value/versions",
)
latest_version = json.loads(rep_schema_versions.content)[-1]

# On récupère le schéma associé à la version la plus récente
rep_schema = requests.get(
    SCHEMA_REGISTRY_URL + f"/subjects/ongoing-trips-value/versions/{latest_version}",
)
trips_schema = json.loads(rep_schema.content)["schema"]

pprint(json.loads(trips_schema))

In [ ]:
import pyspark.sql.functions as func

from pyspark.sql.avro.functions import from_avro

df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_SERVER) \
    .option("subscribe", TOPIC) \
    .option("kafka.group.id", "analytics") \
    .load()

output = df \
    .select(from_avro(func.expr("substring(value, 6, length(value)-5)"), trips_schema).alias("trips"))\
    .select(func.col("trips.*"))\
    .withColumn("dropoff_datetime", func.col("dropoff_datetime").cast("timestamp"))

df_res = output \
    .withWatermark("dropoff_datetime", "2 minutes") \
    .groupBy(
        func.window("dropoff_datetime", "1 minute", "1 minute")
    ) \
    .count()

    